<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Word2Vec-Skipgram-Keras</H1></u></center>

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
import gensim

In [3]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
remove_terms = punctuation + '0123456789'
remove_terms

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'

In [5]:
def preprocessing(text):
    words = word_tokenize(text)
    tokens = [w for w in words if w.lower() not in remove_terms]
    #stopw = stopwords.words('english')
    #tokens = [token for token in tokens if token not in stopw]
    # remove words less than three letters
    #tokens = [word for word in tokens if len(word)>=3]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [6]:
#path = get_file('cosmos.txt', origin="https://www.gutenberg.org/files/8172/8172-0.txt")

In [6]:
corpus = open("../data/History_of_Astronomy.txt", encoding="utf8").readlines()

In [7]:
corpus

['\ufeffThe Project Gutenberg EBook of History of Astronomy, by George Forbes\n',
 '\n',
 '\n',
 'Title: History of Astronomy\n',
 '\n',
 'Author: George Forbes\n',
 '\n',
 '[Illustration: SIR ISAAC NEWTON (From the bust by Roubiliac In Trinity\n',
 'College, Cambridge.)]\n',
 '\n',
 'HISTORY OF ASTRONOMY\n',
 '\n',
 'BY\n',
 '\n',
 'GEORGE FORBES,\n',
 'M.A., F.R.S., M. INST. C. E.,\n',
 '\n',
 "(FORMERLY PROFESSOR OF NATURAL PHILOSOPHY, ANDERSON'S COLLEGE, GLASGOW)\n",
 '\n',
 'AUTHOR OF "THE TRANSIT OF VENUS," RENDU\'S "THEORY OF THE GLACIERS OF\n',
 'SAVOY," ETC., ETC.\n',
 '\n',
 '\n',
 '\n',
 '\n',
 'CONTENTS\n',
 '\n',
 '  PREFACE\n',
 '\n',
 '  BOOK I. THE GEOMETRICAL PERIOD\n',
 '\n',
 '     1. PRIMITIVE ASTRONOMY AND ASTROLOGY\n',
 '\n',
 '     2. ANCIENT ASTRONOMY--CHINESE AND CHALDÆANS\n',
 '\n',
 '     3. ANCIENT GREEK ASTRONOMY\n',
 '\n',
 '     4. THE REIGN OF EPICYCLES--FROM PTOLEMY TO COPERNICUS\n',
 '\n',
 '  BOOK II. THE DYNAMICAL PERIOD\n',
 '\n',
 '     5. DISCOVER

In [8]:
corpus = [preprocessing(sentence) for sentence in corpus if sentence.strip() !='']
corpus

['Project Gutenberg EBook of History of Astronomy by George Forbes',
 'Title History of Astronomy',
 'Author George Forbes',
 'Illustration SIR ISAAC NEWTON From the bust by Roubiliac In Trinity',
 'College Cambridge',
 'HISTORY OF ASTRONOMY',
 'BY',
 'GEORGE FORBES',
 'INST',
 'FORMERLY PROFESSOR OF NATURAL PHILOSOPHY ANDERSON COLLEGE GLASGOW',
 'AUTHOR OF THE TRANSIT OF VENUS RENDU THEORY OF THE GLACIERS OF',
 'SAVOY ETC',
 'CONTENTS',
 'PREFACE',
 'BOOK I THE GEOMETRICAL PERIOD',
 'PRIMITIVE ASTRONOMY AND ASTROLOGY',
 'ANCIENT ASTRONOMY CHINESE AND CHALDÆANS',
 'ANCIENT GREEK ASTRONOMY',
 'THE REIGN OF EPICYCLES FROM PTOLEMY TO COPERNICUS',
 'BOOK II THE DYNAMICAL PERIOD',
 'DISCOVERY OF THE TRUE SOLAR SYSTEM TYCHO BRAHE KEPLER',
 'GALILEO AND THE TELESCOPE NOTIONS OF GRAVITY BY HORROCKS ETC',
 'SIR ISAAC NEWTON LAW OF UNIVERSAL GRAVITATION',
 'NEWTON SUCCESSORS HALLEY EULER LAGRANGE LAPLACE ETC',
 'DISCOVERY OF NEW PLANETS HERSCHEL PIAZZI ADAMS AND LE',
 'VERRIER',
 'BOOK III OBSER

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [10]:
X_train_tokens = tokenizer.texts_to_sequences(corpus)

In [20]:
X_train_tokens

[[1600, 2315, 2316, 2, 179, 2, 57, 9, 535, 893],
 [1256, 179, 2, 57],
 [894, 535, 893],
 [322, 132, 610, 74, 21, 1, 2317, 9, 2318, 6, 2319],
 [1040, 611],
 [179, 2, 57],
 [9],
 [535, 893],
 [2320],
 [1601, 536, 2, 692, 1257, 1258, 1040, 1259],
 [894, 2, 1, 323, 2, 282, 2321, 52, 2, 1, 2322, 2],
 [2323, 779],
 [2324],
 [1602],
 [113, 612, 1, 537, 71],
 [1041, 57, 3, 895],
 [206, 57, 225, 3, 693],
 [206, 613, 57],
 [1, 1042, 2, 1603, 21, 338, 5, 124],
 [113, 1043, 1, 432, 71],
 [66, 2, 1, 145, 56, 61, 102, 140, 84],
 [149, 3, 1, 79, 1604, 2, 481, 9, 896, 779],
 [132, 610, 74, 62, 2, 324, 141],
 [74, 1605, 238, 614, 694, 325, 779],
 [66, 2, 63, 897, 121, 615, 226, 3, 150],
 [283],
 [113, 1044, 33],
 [1260, 2, 780, 326, 2, 1, 56, 61],
 [179, 2, 1, 79, 216],
 [113, 1606, 1, 217, 71],
 [1, 26],
 [1, 45, 3, 897],
 [1261, 3, 1607],
 [1, 1608, 3, 371],
 [2325],
 [1602],
 [29, 616, 28, 23, 51, 6, 27, 898, 5, 482, 1, 617, 2],
 [538, 253, 6, 1, 254, 2, 172, 66, 3],
 [9, 1609, 1, 136, 122, 2, 173, 

In [12]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

4411

In [13]:
items = tokenizer.word_index.items()
items

dict_items([('the', 1), ('of', 2), ('and', 3), ('a', 4), ('to', 5), ('in', 6), ('that', 7), ('it', 8), ('by', 9), ('wa', 10), ('he', 11), ('is', 12), ('be', 13), ('his', 14), ('for', 15), ('with', 16), ('at', 17), ('on', 18), ('this', 19), ('which', 20), ('from', 21), ('star', 22), ('been', 23), ('have', 24), ('but', 25), ('sun', 26), ('these', 27), ('ha', 28), ('an', 29), ('not', 30), ('motion', 31), ('all', 32), ('observation', 33), ('were', 34), ('planet', 35), ('earth', 36), ('one', 37), ('are', 38), ('had', 39), ('or', 40), ('found', 41), ('year', 42), ('great', 43), ('time', 44), ('moon', 45), ('their', 46), ('line', 47), ('so', 48), ('orbit', 49), ('when', 50), ('made', 51), ('theory', 52), ('same', 53), ('first', 54), ('we', 55), ('solar', 56), ('astronomy', 57), ('about', 58), ('also', 59), ('comet', 60), ('system', 61), ('law', 62), ('new', 63), ('astronomer', 64), ('there', 65), ('discovery', 66), ('more', 67), ('no', 68), ('distance', 69), ('they', 70), ('period', 71), ('ot

## Creating the Skipgram Model:

In [17]:
dim_embedddings = 300
inputs = Input(shape=(1, ), dtype='int32')

In [22]:
dim_embedddings = 300

# inputs
inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(vocab_size, dim_embedddings)(inputs)

# context
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(vocab_size, dim_embedddings)(c_inputs)

d = Dot(axes=2)([w, c])

d = Reshape((1,), input_shape=(1, 1))(d)
d = Activation('sigmoid')(d)

model = Model(inputs=[inputs, c_inputs], outputs=d)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 300)       1323300     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 300)       1323300     input_10[0][0]                   
__________________________________________________________________________________________________
dot_3 (Dot

In [23]:
inputs

<tf.Tensor 'input_9:0' shape=(?, 1) dtype=int32>

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam')

## Training the model:

In [ ]:
%%time
n_epochs = 15
for epoch in range(n_epochs):
    loss = 0.
    for i, doc in enumerate(X_train_tokens):
        data, labels = skipgrams(sequence=doc, vocabulary_size=vocab_size, window_size=4)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if x:
            loss += model.train_on_batch(x, y)

    print('Epoch:', epoch, '\tLoss:', loss)

Epoch: 0 	Loss: 1788.638923496008
Epoch: 1 	Loss: 1475.9782007336617
Epoch: 2 	Loss: 1373.8998239412904
Epoch: 3 	Loss: 1262.616573613137
Epoch: 4 	Loss: 1159.170803187415
Epoch: 5 	Loss: 1059.0537358745933
Epoch: 6 	Loss: 977.1972101060674
Epoch: 7 	Loss: 909.7590831201524
Epoch: 8 	Loss: 853.0493990222458
Epoch: 9 	Loss: 807.4501373129897
Epoch: 10 	Loss: 767.2370177389821
Epoch: 11 	Loss: 740.4405831824988
Epoch: 12 	Loss: 718.8705753890099


## Saving the model:

In [ ]:
f = open('../data/word2vec-skipgrams1.txt' ,'w', encoding="utf8")
f.write('{} {}\n'.format(vocab_size-1, dim_embedddings))

weights = model.get_weights()[0]
for word, i in items:
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(weights[i, :])))))
f.close()

## Loading the model:

In [19]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('../data/word2vec-skipgrams1.txt', binary=False)

In [20]:
w2v.most_similar(positive=['solar'])

C:\Anaconda36\envs\tf111_gpu\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gaseous', 0.4597316086292267),
 ('researches', 0.4340711534023285),
 ('harmful', 0.4325682818889618),
 ('physics', 0.4134013056755066),
 ('system', 0.41045206785202026),
 ('news', 0.39892005920410156),
 ('compressed', 0.3597968816757202),
 ('nautical', 0.35958218574523926),
 ('substitute', 0.3580801486968994),
 ('toledo', 0.3563697040081024)]

In [21]:
w2v.most_similar(positive=['system'])

[('advocated', 0.4853593111038208),
 ('solar', 0.4104520082473755),
 ('main', 0.40048810839653015),
 ('relationship', 0.3770222067832947),
 ('answered', 0.3546488583087921),
 ('golden', 0.33952847123146057),
 ('clue', 0.33863192796707153),
 ('harmful', 0.33825457096099854),
 ('driving', 0.3371925354003906),
 ('predominance', 0.33416372537612915)]

In [22]:
w2v.most_similar(positive=['kepler'])

[('wanderer', 0.5605933666229248),
 ('johannes', 0.5162597894668579),
 ('reitlinger', 0.5112121105194092),
 ('independence', 0.4941261410713196),
 ('strassburg', 0.4905438721179962),
 ('yielding', 0.4500398635864258),
 ('slavish', 0.44416677951812744),
 ('contemplated', 0.44190627336502075),
 ('abolish', 0.4218079745769501),
 ('johann', 0.4171288013458252)]

## References:

https://keras.io/preprocessing/sequence/

https://www.tensorflow.org/tutorials/representation/word2vec